# 動態規劃

## 思考順序
1. 這個問題的 base case (最簡單情況) 為何？
2. 這個問題有什麼「`狀態`」？
3. 對於每個「狀態」，可以做出什麼`選擇`，使得「狀態」發生改變？
4. 如何定義 `dp` 陣列/函數的含義，以便表現「狀態」和「選擇」？

## 思路
- `狀態`
- `選擇`
- `dp`陣列的定義。

## 框架
```py
# 初始化 base case
dp[0][0][...] = base case
# 進行狀態轉移
for 狀態1 in 狀態1的所有取值:
    for 狀態2 in 狀態2的所有取值:
        for ...
            dp[狀態1][狀態2][...] = 求極值(選擇1, 選擇2, ...)
```

# 費氏數列 (Fibonacci series)

In [10]:
# Time: O(2^N)
# Space: O(N)
def fib_brute_force(n: int) -> int:
    if n == 0: return 0
    if n == 1 or n == 2: return 1
    return fib_brute_force(n-1) + fib_brute_force(n-2)

fib_brute_force(40)

102334155

## 暴力解最大問題是 重疊子問題

解決重疊子問題的方法

1. Recursion with memo (Up->Bottom)

2. Dynamic Programming (Bottom->Up)

In [17]:
# 1. Recursion with memo (Up->Bottom)
# Time: O(N)
# Space: O(N)
def fib_with_memo(n: int) -> int:
    if n == 0: return 0
    memo = [0 for i in range(n+1)]
    def solver(n: int):
        if n == 1 or n == 2: return 1
        if memo[n] != 0: return memo[n]
        memo[n] = solver(n-1) + solver(n-2)
        return memo[n]
    return solver(n)

fib_with_memo(40)

102334155

In [25]:
# 2. DP
# Time: O(N)
# Space: O(N)
def fib_dp(n: int) -> int:
    if n == 0: return 0
    if n == 1 or n == 2: return 1
    dp = [0 for _ in range(n+1)]
    dp[1] = dp[2] = 1
    for i in range(3, n+1):
        dp[i] = dp[i-1] + dp[i-2] # 狀態n = 狀態n-1 + 狀態n-2
    return dp[n]

fib_dp(40)

102334155

# 狀態壓縮

實際上這個例子中，狀態只有 2 個，不需要一個動態的狀態空間 dp，即可以完成狀態轉移，此時可以再考慮狀態壓縮
- 二維dp -> 一維dp <=> O(N^2) -> O(N)
- 一維dp -> pointers <=> O(N) -> O(1)

In [27]:
# 3. DP with two pointers
# Time: O(N)
# Space: O(1) (狀態壓縮)
def fib_dp_two_pointers(n: int) -> int:
    if n == 0: return 0
    if n == 1 or n == 2: return 1
    current = prev = 1
    for _ in range(3, n+1):
        next = current + prev
        prev = current
        current = next
    return current

fib_dp_two_pointers(40)

102334155

# 湊零錢問題

給出 `k` 種面值硬幣，分別為 `c1, c2, ..., ck`, 每種硬幣的數量無限，試問最少需要幾枚硬幣湊出金額 `amount`？

如果不可能湊出來，返回 `-1`

## 思路框架
1. `Base case`，當 `amount` 為 0 時，演算法回傳 0，因為不需要硬幣就能湊出目標金額。
2. 狀態，確定原問題和子問題的變數：由於硬幣數量無限，硬幣的面額也是動態的，只有目標金額會不斷向 `Base case` 靠近，故狀態只有 `amount` 一個。
3. 選擇，確定狀態轉移，`意即導致狀態產生變化的行為`，只有一種可能會導致目標金額 amount 產生變化，就是幣別的選擇，因此`ck` 為選擇。
4. 明確 `dp` 的定義，`dp(n)：輸入一個目標金額 n，返回湊出目標金額 n 的最少硬幣數`，採用 (Up->Bottom)。

In [49]:
# Brute force
# Time: O(kn^k)
# Space: O(n)
def coin_brute_force(coins: list, amount: int) -> int:
    # 定義：若要湊出金額 m，至少要 dp(n) 枚硬幣
    def dp(n: int) -> int: # O(n^k)
        # base case
        if n == 0: return 0
        if n < 0: return -1
        # 求最小值，初始值為最大值
        res = float('inf')
        for coin in coins: # O(k)
            subproblem = dp(n-coin)
            # 無解，跳過
            if subproblem == -1: continue
            res = min(res, 1+subproblem)
        return res if res != float('inf') else -1
    return dp(amount)

coin_brute_force([1,2,5], 30)

6

In [50]:
# Recursion with memo (Up->Bottom)
# Time: O(kn)
# Space: O(n)
def coin_memo(coins: list, amount: int) -> int:
    memo = [0 for _ in range(amount+1)]
    # 定義：若要湊出金額 m，至少要 dp(n) 枚硬幣
    def dp(n: int) -> int: # O(n^k)
        # 避免重複計算
        if memo[n] != 0: return memo[n]
        # base case
        if n == 0: return 0
        if n < 0: return -1
        # 求最小值，初始值為最大值
        res = float('inf')
        for coin in coins: # O(k)
            subproblem = dp(n-coin)
            # 無解，跳過
            if subproblem == -1: continue
            res = min(res, 1+subproblem)
        memo[n] = res if res != float('inf') else -1
        return memo[n]

    return dp(amount)

coin_memo([1,2,5], 30)

6

In [57]:
# DP (Bottom->Up)
# Time: O(kn)
# Space: O(n)
def coin_dp(coins: list, amount: int) -> int:
    # 求取最小初始狀態須為最大， amount+1，因為全部都用1元硬幣，最多只會有amount枚硬幣。
    dp = [amount+1 for _ in range(amount+1)]
    # base case
    dp[0] = 0
    # for 狀態 in 狀態空間中
    for i in range(len(dp)):
        for coin in coins:
            # 無解
            if i < coin: continue
            # 如果 *選擇* 加入這個 coin，使得硬幣用數較少，就選擇加入。 (狀態轉移)
            dp[i] = min(dp[i], 1+dp[i-coin])
    return dp[amount] if dp[amount] != amount+1 else -1

coin_dp([1,2,5], 30)

6